<a href="https://colab.research.google.com/github/aicuai/Book-SD-MasterGuide/blob/main/ComfyJapan_LTX2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ComfyJapan] ComfyUI LTX-2 動画生成

## 🚀 概要

Lightricks社の最新動画生成モデル **LTX-2** を Google Colab で運用するための統合環境です。ポーズ制御（Pose to Video） と テキスト生成（Text to Video） の両ワークフローに完全対応し、24時間稼働を想定した「自己修復機能」を備えています。

### 🛠️ 技術解説と推奨環境

* **推奨GPU**: **L4 GPU (22.5GB)** 以上。生成速度と安定性の面から **A100 / H100** を強く推奨します。
* **メモリ最適化**: L4 GPUではVRAM容量がモデルサイズ（19B + 12B）に対してシビアなため、自動的にメモリ節約モード（`--lowvram`）を適用します。
* **自己修復ロジック**:
* **モデル配置**: Gemma 3IT 等の複雑な階層モデルを ComfyUI 認識ディレクトリへ自動整理します。
* **自動再起動**: サーバーがメモリ不足（OOM）などでクラッシュした場合、スレッドがそれを検知して Discord に報告し、自動的にプロセスを再立ち上げします。



### 📋 事前準備

1. **HuggingFace**: [LTX-2 Repository](https://huggingface.co/Lightricks/LTX-2) で利用許諾を済ませたアカウントの `HF_TOKEN` を、Colabの「🔑シークレット」に設定してください。
2. **Discord**: 通知用 Webhook URL を `DISCORD_URL` として設定してください。

---


# 🥚各種モデルのDL ＆ ComfyUIの起動🐣
##＜はじめる前の準備＞
### ❶ Google Colab の「シークレット」に `HF_TOKEN` と `DISCORD_URL` を設定してください。
### ❷（▶）ボタンをクリックして起動。完了すると Discord に通知が届きます。


In [ ]:
import os, socket, time, threading, requests, shutil, subprocess, re, sys
from collections import deque
from google.colab import userdata, output
from IPython.display import clear_output

# ==========================================
# 🔑 設定確認 & Discord通知（テスト機能付）
# ==========================================
def get_secret(key):
    try: return userdata.get(key)
    except: return None

hf_token = get_secret('HF_TOKEN')
if hf_token: os.environ['HF_TOKEN'] = hf_token
webhook_url = get_secret('DISCORD_URL')

def send_discord(msg):
    if webhook_url:
        try:
            # タイムアウトを長めに設定し、バックグラウンドでの送信漏れを防ぐ
            requests.post(webhook_url, json={"content": msg}, timeout=15)
        except Exception as e:
            print(f"📡 Discord送信失敗: {e}")

# GPUの正体を暴く
print("🖥️ 物理GPUの正確な情報を確認中...")
try:
    gpu_info = subprocess.check_output(["nvidia-smi", "--query-gpu=name,memory.total", "--format=csv,noheader"]).decode().strip()
except:
    gpu_info = "Unknown GPU"

# 🚀 テスト通知を送信
send_discord(f"🔔 **通知テスト送信**\n💻 検出GPU: `{gpu_info}`\n🛠️ サーバー構築を開始します。")

# ==========================================
# 📁 環境構築 (DWPose ワーニング対策込)
# ==========================================
%cd /content
clear_output()
print(f"🛠️ {gpu_info} 用に環境を構築中...")

# 依存関係（競合を抑制しつつ DWPose 加速用の onnxruntime-gpu を追加）
!pip install -q "protobuf<5.0.0" "huggingface_hub[cli]<1.0"
!pip install -q torch==2.8.0 torchvision==0.23.0 xformers==0.0.32.post1 triton==3.4 requests spandrel onnxruntime-gpu
!pip install -q av albumentations onnx opencv-python

if not os.path.exists("ComfyUI"):
    !git clone https://github.com/Comfy-Org/ComfyUI.git
!pip install -q -r /content/ComfyUI/requirements.txt

%cd /content/ComfyUI/custom_nodes
def clone_node(url):
    name = url.split("/")[-1].replace(".git", "")
    if not os.path.exists(name): subprocess.run(["git", "clone", url], check=False)

# 最新の Comfy-Org 準拠
clone_node("https://github.com/Comfy-Org/ComfyUI-Manager")
clone_node("https://github.com/Lightricks/ComfyUI-LTXVideo")
clone_node("https://github.com/Isi-dev/comfyui_controlnet_aux")
clone_node("https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite")
clone_node("https://github.com/kijai/ComfyUI-KJNodes.git")
clone_node("https://github.com/rgthree/rgthree-comfy.git")

# 各ノードの依存関係を解決
for root, dirs, files in os.walk("/content/ComfyUI/custom_nodes"):
    if "requirements.txt" in files:
        subprocess.run(["pip", "install", "-q", "-r", os.path.join(root, "requirements.txt")], check=False)

%cd /content/ComfyUI
!apt-get -y install -qq aria2

# ========================================
# 📥 モデル準備 (最新推奨 hf download 方式)
# ========================================
print("📥 LTX-2 関連モデルをダウンロード中...")

# 1. Gemma 3 IT 配置修正
!hf download Comfy-Org/ltx-2 split_files/text_encoders/gemma_3_12B_it.safetensors --local-dir models/text_encoders --quiet
g_path = "models/text_encoders/split_files/text_encoders/gemma_3_12B_it.safetensors"
if os.path.exists(g_path): shutil.move(g_path, "models/text_encoders/gemma_3_12B_it.safetensors")

# 2. モデル一括取得
models_to_get = [
    ("Lightricks/LTX-2", "ltx-2-19b-dev-fp8.safetensors", "models/checkpoints"),
    ("Lightricks/LTX-2", "ltx-2-19b-distilled.safetensors", "models/checkpoints"),
    ("Lightricks/LTX-2-19b-IC-LoRA-Pose-Control", "ltx-2-19b-ic-lora-pose-control.safetensors", "models/loras"),
    ("Lightricks/LTX-2-19b-IC-LoRA-Depth-Control", "ltx-2-19b-ic-lora-depth-control.safetensors", "models/loras"),
    ("Comfy-Org/lotus", "lotus-depth-d-v1-1.safetensors", "models/diffusion_models"),
    ("stabilityai/sd-vae-ft-mse-original", "vae-ft-mse-840000-ema-pruned.safetensors", "models/vae"),
    ("Lightricks/LTX-2", "ltx-2-spatial-upscaler-x2-1.0.safetensors", "models/latent_upscale_models")
]
for repo, file, folder in models_to_get:
    if not os.path.exists(os.path.join(folder, file)):
        subprocess.run(["hf", "download", repo, file, "--local-dir", folder, "--quiet"])

# アセット取得 (aria2cによる高速化)
!aria2c --console-log-level=error -c -x 16 -s 16 -d input -o minimalist_sky_view_lounge.png https://huggingface.co/Lightricks/LTX-2/resolve/main/minimalist_sky_view_lounge.png --quiet
!aria2c --console-log-level=error -c -x 16 -s 16 -d input -o minimalist_sky_view_lounge.mp4 https://huggingface.co/Lightricks/LTX-2/resolve/main/minimalist_sky_view_lounge.mp4 --quiet

# ========================================
# 🚀 サービス監視 & クラッシュ解析
# ========================================
restart_count = 0
notified_ready = False

def status_monitor(port):
    global notified_ready
    while True:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(2)
            res = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if res == 0 and not notified_ready:
                proxy_url = output.eval_js(f'google.colab.kernel.proxyPort({port})')
                status = "🎉 ComfyUI 準備完了！" if restart_count == 0 else "🔄 自動再起動に成功"
                msg = f"🚀 **{status}**\n💻 実稼働GPU: `{gpu_info}`\n🔗 URL: {proxy_url}"
                send_discord(msg)
                print(f"\n{msg}\n")
                notified_ready = True
            if res != 0: notified_ready = False
        except: pass
        time.sleep(10)

threading.Thread(target=status_monitor, daemon=True, args=(8188,)).start()

# 起動フラグ判定 (H100/A100はVRAMをフル活用)
mem_flags = "" if any(x in gpu_info for x in ["H100", "A100"]) else "--lowvram"

print(f"🚀 {gpu_info} モードで起動中...")
try:
    while True:
        # Popen でログをリアルタイムスキャン
        process = subprocess.Popen(
            f"python main.py {mem_flags} --cache-none --dont-print-server",
            shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
        )

        log_window = deque(maxlen=20) # 直近20行を保持
        for line in process.stdout:
            print(line, end="")
            log_window.append(line.strip())

            # イベント検知
            if "got prompt" in line.lower():
                send_discord(f"📥 **プロンプトを受理** (GPU: `{gpu_info}`)")
            exec_match = re.search(r"Prompt executed in (\d+\.\d+) seconds", line)
            if exec_match:
                send_discord(f"✅ **生成完了** (所要時間: {exec_match.group(1)}秒)")

        process.wait()
        if process.returncode != 0:
            restart_count += 1
            # クラッシュ原因の診断
            logs_str = "\n".join(log_window)
            reason = "不明なエラー"
            if "OutOfMemoryError" in logs_str: reason = "GPUメモリ不足 (OOM)"
            elif "Killed" in logs_str: reason = "システムによる強制終了 (RAM不足など)"

            report = f"⚠️ **クラッシュを検知しました**\n🔴 推定原因: `{reason}`\n🔄 {restart_count}回目の再起動を行います。\n📝 直近のログ末尾:\n```{log_window[-1]}```"
            send_discord(report)
            time.sleep(5)
        else: break
except KeyboardInterrupt:
    print("⏹️ 停止されました。")

In [ ]:
# @markdown # 👈 統合起動ボタン（H100/A100 診断・通知完全版）
# @markdown ---
# @markdown ※ 起動直後に「テスト通知」がDiscordに届くか確認してください。
# @markdown ※ クラッシュ時は、ログから「OOM（メモリ不足）」等を判定して詳細報告します。

import os, socket, time, threading, requests, shutil, subprocess, re, sys
from collections import deque
from google.colab import userdata, output
from IPython.display import clear_output

# ==========================================
# 🔑 設定確認 & Discordテスト
# ==========================================
def get_secret(key):
    try: return userdata.get(key)
    except: return None

hf_token = get_secret('HF_TOKEN')
if hf_token: os.environ['HF_TOKEN'] = hf_token
webhook_url = get_secret('DISCORD_URL')

def send_discord(msg):
    if webhook_url:
        try:
            requests.post(webhook_url, json={"content": msg}, timeout=10)
        except Exception as e:
            print(f"📡 Discord通知エラー: {e}")

# GPUの正体を暴く
print("🖥️ 現在の物理GPUを確認します...")
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader
gpu_info = subprocess.check_output(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"]).decode().strip()
send_discord(f"🚀 **ComfyUI 起動プロセス開始**\n💻 物理GPU: `{gpu_info}`")

# ==========================================
# 📁 環境構築 (DWPose加速パーツ追加)
# ==========================================
%cd /content
clear_output()
print(f"🛠️ {gpu_info} 用に環境を構築中 (ワーニング対策済)...")

# 依存関係（競合抑制 + 加速用ライブラリ）
!pip install -q "huggingface_hub[cli]<1.0" "protobuf<5.0.0"
!pip install -q torch==2.8.0 torchvision==0.23.0 xformers==0.0.32.post1 triton==3.4 requests spandrel onnxruntime-gpu
!pip install -q av albumentations onnx opencv-python

if not os.path.exists("ComfyUI"):
    !git clone https://github.com/Comfy-Org/ComfyUI.git
!pip install -q -r /content/ComfyUI/requirements.txt

%cd /content/ComfyUI/custom_nodes
def clone_node(url):
    name = url.split("/")[-1].replace(".git", "")
    if not os.path.exists(name): subprocess.run(["git", "clone", url], check=False)

clone_node("https://github.com/Comfy-Org/ComfyUI-Manager")
clone_node("https://github.com/Lightricks/ComfyUI-LTXVideo")
clone_node("https://github.com/Isi-dev/comfyui_controlnet_aux")
clone_node("https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite")
clone_node("https://github.com/kijai/ComfyUI-KJNodes.git")
clone_node("https://github.com/rgthree/rgthree-comfy.git")

# ノードの依存関係解決
for root, dirs, files in os.walk("/content/ComfyUI/custom_nodes"):
    if "requirements.txt" in files:
        subprocess.run(["pip", "install", "-q", "-r", os.path.join(root, "requirements.txt")], check=False)

%cd /content/ComfyUI
!apt-get -y install -qq aria2

# ========================================
# 📥 モデル準備 (最新 hf download 方式)
# ========================================
print("📥 LTX-2 関連モデルを最新方式で準備中...")

# Gemma 3 IT 配置修正 (Text Encoder)
!hf download Comfy-Org/ltx-2 split_files/text_encoders/gemma_3_12B_it.safetensors --local-dir models/text_encoders --quiet
g_wrong = "models/text_encoders/split_files/text_encoders/gemma_3_12B_it.safetensors"
if os.path.exists(g_wrong): shutil.move(g_wrong, "models/text_encoders/gemma_3_12B_it.safetensors")

# 一括モデル取得
models = [
    ("Lightricks/LTX-2", "ltx-2-19b-dev-fp8.safetensors", "models/checkpoints"),
    ("Lightricks/LTX-2", "ltx-2-19b-distilled.safetensors", "models/checkpoints"), # 蒸留版
    ("Lightricks/LTX-2-19b-IC-LoRA-Pose-Control", "ltx-2-19b-ic-lora-pose-control.safetensors", "models/loras"),
    ("Lightricks/LTX-2-19b-IC-LoRA-Depth-Control", "ltx-2-19b-ic-lora-depth-control.safetensors", "models/loras"),
    ("Comfy-Org/lotus", "lotus-depth-d-v1-1.safetensors", "models/diffusion_models"),
    ("stabilityai/sd-vae-ft-mse-original", "vae-ft-mse-840000-ema-pruned.safetensors", "models/vae"),
    ("Lightricks/LTX-2", "ltx-2-spatial-upscaler-x2-1.0.safetensors", "models/latent_upscale_models")
]
for repo, file, folder in models:
    if not os.path.exists(os.path.join(folder, file)):
        subprocess.run(["hf", "download", repo, file, "--local-dir", folder, "--quiet"])

# アセット
!aria2c --console-log-level=error -c -x 16 -s 16 -d input -o minimalist_sky_view_lounge.png https://huggingface.co/Lightricks/LTX-2/resolve/main/minimalist_sky_view_lounge.png --quiet
!aria2c --console-log-level=error -c -x 16 -s 16 -d input -o minimalist_sky_view_lounge.mp4 https://huggingface.co/Lightricks/LTX-2/resolve/main/minimalist_sky_view_lounge.mp4 --quiet

# ========================================
# 🚀 サーバー監視 & クラッシュ解析
# ========================================
restart_count = 0
notified_ready = False

def status_monitor(port):
    global notified_ready
    while True:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            res = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if res == 0 and not notified_ready:
                proxy_url = output.eval_js(f'google.colab.kernel.proxyPort({port})')
                msg = f"🎉 **ComfyUI 準備完了！**\n💻 GPU: `{gpu_info}`\n🔗 URL: {proxy_url}"
                send_discord(msg)
                print(f"\n{msg}\n")
                notified_ready = True
            if res != 0: notified_ready = False
        except: pass
        time.sleep(10)

threading.Thread(target=status_monitor, daemon=True, args=(8188,)).start()

# メインループ (クラッシュ解析付)
mem_flags = "" if any(x in gpu_info for x in ["H100", "A100"]) else "--lowvram"
print(f"🚀 {gpu_info} 高速モードで起動中...")

try:
    while True:
        process = subprocess.Popen(
            f"python main.py {mem_flags} --cache-none --dont-print-server",
            shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
        )

        log_window = deque(maxlen=15) # 直近15行のログを保持
        for line in process.stdout:
            print(line, end="")
            log_window.append(line.strip())

            # 正常イベント
            if "got prompt" in line.lower():
                send_discord(f"📥 **プロンプトを受理** (GPU: `{gpu_info}`)")

            exec_match = re.search(r"Prompt executed in (\d+\.\d+) seconds", line)
            if exec_match:
                send_discord(f"✅ **動画生成完了！** (所要時間: {exec_match.group(1)}秒)")

        process.wait()
        if process.returncode != 0:
            restart_count += 1
            # クラッシュ原因の推測
            reason = "不明なエラー"
            logs_str = "\n".join(log_window)
            if "OutOfMemoryError" in logs_str: reason = "GPUメモリ不足 (OOM)"
            elif "Segmentation fault" in logs_str: reason = "セグメンテーション違反 (メモリ不整合)"
            elif "Killed" in logs_str: reason = "システムによる強制終了 (RAM不足等)"

            error_report = f"⚠️ **クラッシュを検知しました**\n🔴 原因: `{reason}`\n🔄 {restart_count}回目の再起動を行います。\n📝 直近のログ:\n```{log_window[-1]}```"
            send_discord(error_report)
            time.sleep(5)
        else: break
except KeyboardInterrupt:
    print("⏹️ 停止されました。")

# 技術的な改善点まとめ
## 技術解説：2026年版 LTX-2 運用ガイド

この環境では、以下の高度な最適化が施されています。

VRAM マネジメント: A100 80GB ではモデル全体を VRAM に保持する高速推論を、L4 22.5GB では --lowvram を用いた堅実な動作を自動的に選択します。

アセット不整合の解消: ワークフローが要求する woman_with_paper 系のファイルを、現在の正しい配布URLから自動取得します。

マルチリポジトリ・ダウンロード: 単一の配布元では揃わない Pose Control LoRA や Text Encoder を、複数の Hugging Face リポジトリを横断して一括で揃えます。

これで、準備は万全です。提供された JSON ファイル を読み込み、LTX-2 の驚異的な動画生成能力をご体験ください！
# 謝辞：このノートブックは AICU Japan がメンテナンスしております。質問は [X@AICUai](https://x.com/AICUai) まで。